In [ ]:
import numpy as np
from graph_setup import G as newG

In [ ]:
node_name1 = "PVH-937"
node_name2 = "PVH-941"


mark1 = newG.nodes[node_name1]["markers"].copy()
mark2 = newG.nodes[node_name2]["markers"].copy()


common_markers = mark1.merge(mark2, on="Mkr", suffixes=('_mark1', '_mark2'))

common_markers.rename(columns={"Depth (meters)_mark1": node_name1, "Depth (meters)_mark2": node_name2})
print(common_markers)


In [ ]:
from calc_functions import optimal_window
mark1_n1 = 1075.1807
mark2_n1 = 1290.3693

mark1_n2 = 1073.5878
mark2_n2 = 1290.3293

m_tol = 5

bounds = [(mark1_n1-m_tol, mark1_n1+m_tol), (mark2_n1-m_tol, mark2_n1+m_tol), (mark1_n2-m_tol, mark1_n2+m_tol), (mark2_n2-m_tol, mark2_n2+m_tol)]
print(bounds)

opt, error = optimal_window(newG, node_name1, node_name2, bounds)

print(f"Optimal windows: {opt}")
print(f"Minimum mse: {error}")




In [ ]:
from calc_functions import windows_test
windows_test(newG, node_name1, node_name2, opt, show_plots=True)